In [34]:
#import libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.model_selection import TimeSeriesSplit

In [35]:
##Data Preparation
# preparing independent and dependent features
def prepare_data(tx_cases, n_features):
	X, y =[],[]
	for i in range(len(tx_cases)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(tx_cases)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = tx_cases[i:end_ix], tx_cases[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [36]:
#define input sequence
tx_cases = pd.read_csv("Texas_covid_data.csv", usecols=["cases"])
#print(tx_cases)
#indicate number of features
n_steps = 3
# split into samples
X, y = prepare_data(tx_cases.values, n_steps)

In [37]:
#inspect the shape of X
#(number of rows, number of columns)
X.shape

(163, 3, 1)

In [38]:
# X is in 2 dimensions. For a LSTM model X needs to be in 3 dimensions
#reshape X by adding the dimension 'n_element'
n_element = 1
X = X.reshape((X.shape[0], X.shape[1], n_element))

In [39]:
##Model Building
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_element)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [40]:
#split X, y variables into training & test data sets for each variable
tscv = TimeSeriesSplit()
print(tscv)
TimeSeriesSplit(max_train_size=None, n_splits=5)
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TimeSeriesSplit(max_train_size=None, n_splits=3)
('TRAIN:', array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42]), 'TEST:', array([43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
       60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
       77, 78, 79, 80, 81, 82]))
('TRAIN:', array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]), 'TEST:', array([ 83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111

In [41]:
##Model Training
#train using training data set
model.fit(X_train, y_train, epochs=30, verbose=1)

Train on 123 samples
Epoch 1/30
123/123 [==============================] - 3s 21ms/sample - loss: 1410548682.9268
Epoch 2/30
123/123 [==============================] - 0s 286us/sample - loss: 1193335205.4634
Epoch 3/30
123/123 [==============================] - 0s 294us/sample - loss: 1045461614.8293
Epoch 4/30
123/123 [==============================] - 0s 262us/sample - loss: 925639199.7398
Epoch 5/30
123/123 [==============================] - 0s 211us/sample - loss: 713581473.3008
Epoch 6/30
123/123 [==============================] - 0s 204us/sample - loss: 453459628.2276
Epoch 7/30
123/123 [==============================] - 0s 221us/sample - loss: 327487058.9919
Epoch 8/30
123/123 [==============================] - 0s 243us/sample - loss: 112566399.9350
Epoch 9/30
123/123 [==============================] - 0s 226us/sample - loss: 8737591.8232
Epoch 10/30
123/123 [==============================] - 0s 225us/sample - loss: 303040776.8455
Epoch 11/30
123/123 [===========================

In [42]:
##Model Evaluation
#evaluate using test data set
model.evaluate(x=X_test, y=y_test, verbose=1)

40/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

387137158.4

In [43]:
##Model Prediction
#predict using numpy array of daily COVID-19 cases in Texas
model.predict(X.astype('float32'))

array([[7.65908420e-01],
       [9.81279850e-01],
       [9.81279850e-01],
       [9.81279850e-01],
       [9.81279850e-01],
       [9.81279850e-01],
       [9.81279850e-01],
       [1.18229723e+00],
       [1.60862887e+00],
       [2.32638574e+00],
       [3.31026149e+00],
       [5.81760168e+00],
       [1.08678312e+01],
       [1.08678312e+01],
       [1.11051311e+01],
       [1.17142754e+01],
       [1.27396326e+01],
       [1.27396326e+01],
       [1.27396326e+01],
       [1.29725771e+01],
       [1.44404116e+01],
       [1.85162601e+01],
       [2.48953362e+01],
       [2.93592453e+01],
       [3.20346527e+01],
       [3.91123047e+01],
       [4.46207199e+01],
       [5.62804260e+01],
       [6.39331932e+01],
       [8.01533203e+01],
       [1.03624252e+02],
       [1.18116982e+02],
       [1.31519287e+02],
       [1.31792557e+02],
       [1.48696030e+02],
       [2.15876724e+02],
       [3.13226410e+02],
       [4.52315582e+02],
       [5.85622986e+02],
       [6.80939209e+02],
